In [99]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import copy
%matplotlib inline
from sklearn import preprocessing

In [100]:
import warnings
warnings.filterwarnings(action='ignore')

In [101]:
plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus']=False

In [102]:
temp201904 = pd.read_csv("/Users/jaeyoungcho/Desktop/2020 금융 빅데이터 챌린지-공모전데이터/데이터/비씨카드/거래내역_201904.csv", sep=",", encoding="CP949")
temp201905 = pd.read_csv("/Users/jaeyoungcho/Desktop/2020 금융 빅데이터 챌린지-공모전데이터/데이터/비씨카드/거래내역_201905.csv", sep=",", encoding="CP949")
temp202004 = pd.read_csv("/Users/jaeyoungcho/Desktop/2020 금융 빅데이터 챌린지-공모전데이터/데이터/비씨카드/거래내역_202004.csv", sep=",", encoding="CP949")
temp202005 = pd.read_csv("/Users/jaeyoungcho/Desktop/2020 금융 빅데이터 챌린지-공모전데이터/데이터/비씨카드/거래내역_202005.csv", sep=",", encoding="CP949")
trade_online = pd.read_excel("/Users/jaeyoungcho/Desktop/2020 금융 빅데이터 챌린지-공모전데이터/데이터/비씨카드/온라인구매품목.xlsx", sep=",", encoding="CP949")

### 201904~202005 까지의 데이터는 갯수를 맞춰서 잘라준 듯하다.

### 기본적인 NaN값, 중복값 조회 및 수정

In [104]:
temp201904.drop(['할부개월수'],axis = 1,inplace = True)
temp201905.drop(['할부개월수'],axis = 1,inplace = True)

In [105]:
trade_online = trade_online.drop(columns=trade_online.columns[6:])

In [106]:
trade_online.isnull().sum()

기준년월      0
연령코드      0
성별코드      0
소분류코드     0
소분류명     24
총매출건수     0
dtype: int64

In [107]:
trade_online.dropna(inplace=True)
trade_online.isnull().sum()

기준년월     0
연령코드     0
성별코드     0
소분류코드    0
소분류명     0
총매출건수    0
dtype: int64

# 매출금액 예상
> 기대효과 : 매출금액 예출 높은 사람에게 더 좋은 카드, 더 좋은 혜택을 통해 기존 회원들의 이탈을 방지할 수 있음.

> 2019년의 데이터를 이용해 2020년의 데이터를 예측할 예정

> 19년도 매출금액을 7등급으로 나누고 replace

> 해당 데이터 내에서 train,test 로 나눠 정확도 값 확인

> 적절한 정확도가 나오면 이후 20년도 에 적용

> 같은 feature 들로 20년도 데이터에도 정확도 확인

In [108]:
temp_19 = pd.concat((temp201904,temp201905), axis = 0)
temp_20 = pd.concat((temp202004,temp202005), axis = 0)

In [109]:
temp_19.reset_index(drop=True,inplace = True)
temp_20.reset_index(drop=True,inplace = True)

In [111]:
sex = {'M':0,"F":1}
temp_19.replace(sex, inplace = True)
temp_20.replace(sex, inplace = True)

### 19년도 카드데이터에서 중복값 확인

In [114]:
temp_19.loc[temp_19.duplicated(subset=temp_19.columns)]

,기준일자,가맹점신우편번호,고객신우편번호,생년,연령대코드,성별코드,매출구분코드,가맹점업종코드,매출금액
26113,20190401,8393,12104,1977,40,1,5,4076,187735
26120,20190401,8393,12910,1972,45,0,5,4076,317530
26121,20190401,8393,12910,1977,40,1,5,4076,289392
26122,20190401,8393,12910,1981,35,1,5,4076,283328
26126,20190401,6654,62281,1997,20,0,5,9998,13190
...,...,...,...,...,...,...,...,...,...
2097131,20190514,13606,14600,1958,60,1,5,9153,101500
2097133,20190514,13606,14600,1966,50,0,5,9153,84990
2097134,20190514,13606,14600,1967,50,1,5,9153,90473
2097139,20190514,13606,14613,1974,45,0,5,9153,35254


In [115]:
temp_19.iloc[26113]

기준일자        20190401
가맹점신우편번호        8393
고객신우편번호        12104
생년              1977
연령대코드             40
성별코드               1
매출구분코드             5
가맹점업종코드         4076
매출금액          187735
Name: 26113, dtype: int64

In [116]:
temp_19.loc[(temp_19[temp_19.columns[0]]==20190401) & (temp_19[temp_19.columns[1]]==8393)  & (temp_19[temp_19.columns[2]]==12104)  & (temp_19[temp_19.columns[3]]==1977)  & (temp_19[temp_19.columns[4]]==40)  & (temp_19[temp_19.columns[5]]==0)  & (temp_19[temp_19.columns[6]]==5)  & (temp_19[temp_19.columns[7]]==4076)  & (temp_19[temp_19.columns[8]]==187735)]

,기준일자,가맹점신우편번호,고객신우편번호,생년,연령대코드,성별코드,매출구분코드,가맹점업종코드,매출금액


## 중복데이터가 2097150개 중에서 394268개가 발견됨

#### feature 들의 모든 값이 일치할 수 있는 확률이 존재하지만 그럼에도 불구하고 18.8%, 거의 20%에 달하는 데이터가 중복된다는 건 맞춰주기 위해 의도적으로 늘려 놓은 것으로 확인됨

In [117]:
temp_19.drop_duplicates(inplace= True)

In [118]:
temp_19.reset_index(drop=True,inplace = True)

In [119]:
temp_19

,기준일자,가맹점신우편번호,고객신우편번호,생년,연령대코드,성별코드,매출구분코드,가맹점업종코드,매출금액
0,20190401,6654,14688,1995,25,0,5,9998,128180
1,20190401,6654,14688,1998,20,1,5,9998,14620
2,20190401,6654,14697,1999,20,0,5,9998,32000
3,20190401,6654,14699,1960,60,1,5,9998,17588
4,20190401,6654,14699,1990,30,1,5,9998,41463
...,...,...,...,...,...,...,...,...,...
1702877,20190514,13606,14619,1976,40,1,5,9153,103467
1702878,20190514,13606,14620,1964,55,1,5,9153,106097
1702879,20190514,13606,14621,1958,60,1,5,9153,70097
1702880,20190514,13606,14633,1994,25,1,5,9153,196167


#### 2020년 데이터에는 중복값이 하나도 발견되지 않음. 2019년데이터를 2020년 데이터에 맞춰주기 위해 의도적으로 늘린 것 같다는 의심이 거의 확실시됨

In [120]:
temp_20.loc[temp_20.duplicated(subset=temp_20.columns)]

,기준일자,가맹점신우편번호,고객신우편번호,생년,연령대코드,성별코드,매출구분코드,가맹점업종코드,할부개월수,매출금액


# 유동인구 데이터 추가해주기

In [121]:
move_1904 = pd.read_csv('/Users/jaeyoungcho/Desktop/2020 금융 빅데이터 챌린지-공모전데이터/데이터/케이티/유동인구일집계_201904.csv',sep = ',',encoding='CP949')
move_1905 = pd.read_csv('/Users/jaeyoungcho/Desktop/2020 금융 빅데이터 챌린지-공모전데이터/데이터/케이티/유동인구일집계_201905.csv',sep = ',',encoding='CP949')
move_2004 = pd.read_csv('/Users/jaeyoungcho/Desktop/2020 금융 빅데이터 챌린지-공모전데이터/데이터/케이티/유동인구일집계_202004.csv',sep = ',',encoding='CP949')
move_2005 = pd.read_csv('/Users/jaeyoungcho/Desktop/2020 금융 빅데이터 챌린지-공모전데이터/데이터/케이티/유동인구일집계_202005.csv',sep = ',',encoding='CP949') 

In [122]:
move_19 = pd.concat((move_1904,move_1905),axis = 0)
move_20 = pd.concat((move_2004,move_2005),axis = 0)

move_19.reset_index(drop=True,inplace = True)
move_20.reset_index(drop=True,inplace = True)

move_19['연령대구분값'] = move_19['연령대구분값'].apply(lambda x: int(str(x)[-2:]))

In [123]:
# 유동인구의 연령대구분값과 카드데이터의 연령대구분값이 다르다. 카드데이터에 맞출 것이기 때문에 유동인구 0~10 drop

print(sorted(move_19['연령대구분값'].unique()))

print(sorted(temp_19['연령대코드'].unique()))

[0, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70]
[10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70]


In [124]:
move_19.drop(move_19[move_19['연령대구분값'] <= 10].index , axis = 0 ,inplace = True)

move_19[move_19['연령대구분값'] == 0]

move_19.drop(['24시간코드'],axis = 1,inplace = True)

move_19.rename(columns={'연령대구분값':'연령대코드'},inplace = True)

move_19.replace({'M':0,"F":1},inplace = True)

move_19.drop_duplicates(inplace = True)

move_19.reset_index(drop = True,inplace = True)

In [125]:
move_19

,기준일자,성별코드,연령대코드,행정동번호,인구수
0,20190401,0,65,11110710,37
1,20190401,1,35,11110700,61
2,20190401,1,45,11110580,58
3,20190401,1,55,11110550,137
4,20190401,0,70,11110700,66
...,...,...,...,...,...
550824,20190510,0,20,11110515,252
550825,20190510,1,60,11110630,382
550826,20190510,1,15,11110650,285
550827,20190510,0,55,11110630,349


In [129]:
temp_19

,기준일자,가맹점신우편번호,고객신우편번호,생년,연령대코드,성별코드,매출구분코드,가맹점업종코드,매출금액
0,20190401,6654,14688,1995,25,0,5,9998,128180
1,20190401,6654,14688,1998,20,1,5,9998,14620
2,20190401,6654,14697,1999,20,0,5,9998,32000
3,20190401,6654,14699,1960,60,1,5,9998,17588
4,20190401,6654,14699,1990,30,1,5,9998,41463
...,...,...,...,...,...,...,...,...,...
1702877,20190514,13606,14619,1976,40,1,5,9153,103467
1702878,20190514,13606,14620,1964,55,1,5,9153,106097
1702879,20190514,13606,14621,1958,60,1,5,9153,70097
1702880,20190514,13606,14633,1994,25,1,5,9153,196167


In [130]:
card_19 = pd.merge(temp_19,move_19,on = ['기준일자','연령대코드','성별코드'])

In [131]:
card_19

,기준일자,가맹점신우편번호,고객신우편번호,생년,연령대코드,성별코드,매출구분코드,가맹점업종코드,매출금액,행정동번호,인구수
0,20190401,6654,14688,1995,25,0,5,9998,128180,11110690,25
1,20190401,6654,14688,1995,25,0,5,9998,128180,11110630,687
2,20190401,6654,14688,1995,25,0,5,9998,128180,11110630,541
3,20190401,6654,14688,1995,25,0,5,9998,128180,11110540,99
4,20190401,6654,14688,1995,25,0,5,9998,128180,11110640,646
...,...,...,...,...,...,...,...,...,...,...,...
645774306,20190514,13606,12911,1949,70,0,5,9153,96483,11110700,160
645774307,20190514,13606,12911,1949,70,0,5,9153,96483,11110650,47
645774308,20190514,13606,12911,1949,70,0,5,9153,96483,11110515,170
645774309,20190514,13606,12911,1949,70,0,5,9153,96483,11110650,150
